In [67]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy
import pickle
from glob import glob

"""Change to the data folder"""
new_path = "./new_train/new_train"
test_path = './new_val_in/new_val_in'
# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [68]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)
test_dataset = ArgoverseDataset(data_path=test_path)

### Create a loader to enable batch processing

In [79]:
batch_sz = 4

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
# #     inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
#     inp = [numpy.dstack([scene['p_in']]) for scene in batch]
# #     out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
#     out = [numpy.dstack([scene['p_out']]) for scene in batch]
    inp = [numpy.dstack([scene['p_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:], scene['v_in'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
#     out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:], scene['v_out'][scene['track_id'][:,0,0]==scene['agent_id'],:,:]]) for scene in batch]
    inp = torch.Tensor(inp)
    out = torch.Tensor(out)
    return [inp, out]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

test_loader = DataLoader(test_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

### Visualize the batch of sequences

In [70]:
import matplotlib.pyplot as plt
import random

agent_id = 0

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])

In [71]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class NN(nn.Module):
    """This class defines your deep learning model that extends a Module class
      The constructor of your class defines the layers of the model. 
      The forward() function defines how to forward propagate 
      input through the defined layers of the model.

    """
    def __init__(self):
        super(NN, self).__init__()
#         self.flatten = nn.Flatten()
        self.LSTM_stack = nn.Sequential(
            nn.LSTM(input_size=4, hidden_size=4, num_layers=1, batch_first=True),
        )
        self.LSTM_many = nn.LSTM(input_size=4, hidden_size=4, num_layers=1, batch_first=True)

    def forward(self, x):
#         print(x.shape)
        x = self.LSTM_stack(x)
#         x = torch.reshape(x, (4,60,76))
#         x = x.float()
#         print(x[0][:,-1,:])
        output = x[0][:,-1,:].reshape((-1,1,4))
#         print(output.shape)
#         print(x[0][:,-1,:].reshape((4,1,2)))
        for i in range(29):
            x = self.LSTM_many(x[0][:,-1,:].reshape((-1,1,4),x[1]))
            output = torch.cat((output, x[0]), 1)
#             print(output.shape)
#         print(output.shape)
        return output

In [72]:
print(NN())

NN(
  (LSTM_stack): Sequential(
    (0): LSTM(4, 4, batch_first=True)
  )
  (LSTM_many): LSTM(4, 4, batch_first=True)
)


In [73]:
from tqdm import tqdm_notebook as tqdm

def train(model, device, train_loader, optimizer, epoch, log_interval=10000):
    model.train()
    iterator = tqdm(train_loader, total=int(len(train_loader)))
    counter = 0
    MSELoss = nn.MSELoss()
    for i_batch, sample_batch in enumerate(train_loader):
        inp, out = sample_batch
#         print("myinp", inp.shape)
#         print("mytryinp", inp[:,0,:,:].shape)
#         print("myout", out.shape)
#         print("mytryout", out[:,0,:,:].shape)
        inp = inp[:,0,:,:]
        out = out[:,0,:,:]
        """TODO:
          Deep learning model
          training routine
        """
        data, target = inp.to(device), out.to(device)
        optimizer.zero_grad()
        output = model(data)
        #print(output[0])
#         print("OURPUT", output[0].shape)
#         print("TARGET", target.shape)
        #target = torch.argmax(target, 1)
        loss = MSELoss(output, target)
        loss.backward()
        optimizer.step()
        counter += 1
        iterator.set_postfix(loss=(loss.item()*data.size(0) / (counter * train_loader.batch_size)))
#         print(out.shape)
#         show_sample_batch(sample_batch, agent_id)
#         break

In [80]:
device = "cuda"
model = NN().to(device) #using gpu here
optimizer = optim.Adagrad(model.parameters(), lr=0.01, lr_decay=0, weight_decay=0, initial_accumulator_value=0)
num_epoch = 2

for epoch in range(1, num_epoch + 1):
        train(model, device, val_loader, optimizer, epoch)
#         test(model, device, test_loader)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


KeyboardInterrupt: 

In [ ]:
def predict(model, device, test_loader):
    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            

## everything below is trying to get the outputs

In [32]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = "./new_val_in/new_val_in"
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
validation_dataset  = ArgoverseDataset(data_path=new_path)


In [44]:
validation_dataset[0]['p_in'][validation_dataset[0]['track_id'][:,0,0]==validation_dataset[0]['agent_id'],:,0]

array([[1697.71948242, 1698.5793457 , 1699.5020752 , 1700.39123535,
        1701.33813477, 1702.20678711, 1702.95654297, 1703.99707031,
        1704.39428711, 1705.8359375 , 1706.51916504, 1706.89245605,
        1708.3371582 , 1709.33251953, 1709.64978027, 1710.62915039,
        1712.06347656, 1712.99511719, 1713.8626709 ]])

In [47]:
#validation_dataset[0]
# x positions for on pkl file
valid_array = []
for data in validation_dataset:
    valid_array.append(data['p_in'][data['track_id'][:,0,0]==data['agent_id'],:,0].flatten())
    valid_array.append(data['p_in'][data['track_id'][:,0,0]==data['agent_id'],:,1].flatten())

In [49]:
len(valid_array[0])

19

In [56]:

batch_sz = 4

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
#     inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    inp = [numpy.dstack([scene['p_in']]) for scene in batch]
#     out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    #out = [numpy.dstack([scene['p_out']]) for scene in batch]
    inp = torch.Tensor(inp)
    #out = torch.Tensor(out)
    return [inp]

validation_loader = DataLoader(valid_array,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)



In [64]:
validation_loader

In [66]:
for i_batch, sample_batch in enumerate(validation_loader):
    print(i_batch)
    inp = sample_batch[0]
#         print("myinp", inp.shape)
#         print("mytryinp", inp[:,0,:,:].shape)
#         print("myout", out.shape)
    #inp = inp[:,0,:,:]
    #out = out[:,0,:,:]
    """TODO:
       Deep learning model
       training routine
    """
    data = inp.to(device)
    output = model(data)
#validation_loader = validation_loader[:,0,:,:]
#output = model(validation_loader)

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [55]:
output = model(valid_array)

AttributeError: 'list' object has no attribute 'size'